## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server (where you can look at6 the experimental progress in RT if needed)
- backend store: sqlite database (will allow model registry)
- artifacts store: local filesystem (or cloud)

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [12]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [13]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [18]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1750727650487, experiment_id='1', last_update_time=1750727650487, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1750727618785, experiment_id='0', last_update_time=1750727618785, lifecycle_stage='active', name='Default', tags={}>]

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/06/24 01:15:23 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/21d53e6ff6af4e0197ed6e5df3bfef4e/artifacts'
🏃 View run receptive-perch-919 at: http://127.0.0.1:5000/#/experiments/1/runs/21d53e6ff6af4e0197ed6e5df3bfef4e
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


In [20]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1750727650487, experiment_id='1', last_update_time=1750727650487, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/0', creation_time=1750727618785, experiment_id='0', last_update_time=1750727618785, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [21]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [22]:
client.search_registered_models()

[]

In [ ]:
#Register a model
run_id = client.search_runs(experiment_ids='1')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2025/06/24 01:15:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1750727757313, current_stage='None', description='', last_updated_timestamp=1750727757313, name='iris-classifier', run_id='21d53e6ff6af4e0197ed6e5df3bfef4e', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/21d53e6ff6af4e0197ed6e5df3bfef4e/artifacts/models', status='READY', status_message=None, tags={}, user_id='', version='1'>